In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir("D:\\빅데이터\\Google 머신러닝 스터디잼\\titanic")

In [3]:
train = pd.read_csv("train.csv")

In [4]:
test = pd.read_csv("test.csv")

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [93]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_Cabin,NEW_Age,NEW_Fare
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,2030.0,7.8292
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,4050.0,7.0000
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,6070.0,9.6875
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,2030.0,8.6625
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,2030.0,12.2875


In [94]:
# PassengerID & Name 열 삭제
del train['PassengerId']
del train['Name']
del test['PassengerId']
del test['Name']

columns & na 개수 확인

In [85]:
train.info() # age, cabin, embarked에 null 값 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
Survived        891 non-null int64
Pclass          891 non-null int64
Sex             891 non-null object
Age             714 non-null float64
SibSp           891 non-null int64
Parch           891 non-null int64
Ticket          891 non-null object
Fare            891 non-null float64
Cabin           204 non-null object
Embarked        889 non-null object
NEW_Cabin       891 non-null int64
NEW_Embarked    891 non-null object
NEW_Age         891 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 90.6+ KB


In [92]:
test.info() # age, fare, cabin에 null 값 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
NEW_Cabin      418 non-null int64
NEW_Age        418 non-null float64
NEW_Fare       418 non-null float64
dtypes: float64(4), int64(5), object(5)
memory usage: 45.8+ KB


NA 값 채우기 - 이전 데이터 비율 고려

In [24]:
# Cabin NA 채우기
train['NEW_Cabin'] = pd.isna(train['Cabin']).map({True:0, False:1})
test['NEW_Cabin'] = pd.isna(test['Cabin']).map({True:0, False:1})

In [95]:
# Embarked NA 채우기
C = len(train[train['Embarked']=='C'])/(pd.isna(train['Embarked'])==False).sum()
Q = len(train[train['Embarked']=='Q'])/(pd.isna(train['Embarked'])==False).sum()
S = len(train[train['Embarked']=='S'])/(pd.isna(train['Embarked'])==False).sum()
train['NEW_Embarked'] = train['Embarked'].fillna(np.random.choice(['C','Q','S'], p=[C,Q,S]))

In [82]:
# Age NA 채우기
def age_split(x):
    if x < 20:
        return 10
    elif x < 40:
        return 2030
    elif x < 60:
        return 4050
    elif x < 80:
        return 6070
    else:
        return np.nan
    
train['NEW_Age'] = train['Age'].map(lambda x: age_split(x))
test['NEW_Age'] = test['Age'].map(lambda x: age_split(x))
age_010 = len(train[train['NEW_Age']==10])/(pd.isna(train['NEW_Age'])==False).sum()
age_2030 = len(train[train['NEW_Age']==2030])/(pd.isna(train['NEW_Age'])==False).sum()
age_4050 = len(train[train['NEW_Age']==4050])/(pd.isna(train['NEW_Age'])==False).sum()
age_6070 = len(train[train['NEW_Age']==6070])/(pd.isna(train['NEW_Age'])==False).sum()
train['NEW_Age'] = train['NEW_Age'].fillna(np.random.choice([10,2030,4050,6070], p =[age_010, age_2030, age_4050, age_6070]))
test['NEW_Age'] = test['NEW_Age'].fillna(np.random.choice([10,2030,4050,6070], p =[age_010, age_2030, age_4050, age_6070]))

In [204]:
# Fare NA 채우기
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

범주형 변수(Pclass, Sex, Ticket, Embarked, NEW_Age) 처리

In [211]:
set(train['Pclass'])

{1, 2, 3}

In [212]:
# Pclass one-hot encoding
train['NEW_Pclass_1'] = train['Pclass'].map({1:1, 2:0, 3:0})
train['NEW_Pclass_2'] = train['Pclass'].map({1:0, 2:1, 3:0})
train['NEW_Pclass_3'] = train['Pclass'].map({1:0, 2:0, 3:1})
test['NEW_Pclass_1'] = test['Pclass'].map({1:1, 2:0, 3:0})
test['NEW_Pclass_2'] = test['Pclass'].map({1:0, 2:1, 3:0})
test['NEW_Pclass_3'] = test['Pclass'].map({1:0, 2:0, 3:1})

In [117]:
# Sex one-hot encoding
train['NEW_Sex'] = train['Sex'].map({'male':0,'female':1})
test['NEW_Sex'] = test['Sex'].map({'male':0,'female':1})

In [101]:
# Ticket...은 일단 사용하지 말자...!
train['Ticket'].iloc[0:4]

0           A/5 21171
1            PC 17599
2    STON/O2. 3101282
3              113803
Name: Ticket, dtype: object

In [106]:
# Embarked one-hot encoding
train['NEW_Embarked_C'] = train['NEW_Embarked'].map({'C':1,'Q':0,'S':0})
train['NEW_Embarked_Q'] = train['NEW_Embarked'].map({'C':0,'Q':1,'S':0})
train['NEW_Embarked_S'] = train['NEW_Embarked'].map({'C':0,'Q':0,'S':1})
test['NEW_Embarked_C'] = test['Embarked'].map({'C':1,'Q':0,'S':0})
test['NEW_Embarked_Q'] = test['Embarked'].map({'C':0,'Q':1,'S':0})
test['NEW_Embarked_S'] = test['Embarked'].map({'C':0,'Q':0,'S':1})

In [109]:
# Age one-hot encoding
train['NEW_Age_010'] = train['NEW_Age'].map({10:1, 2030:0, 4050:0, 6070:0})
train['NEW_Age_2030'] = train['NEW_Age'].map({10:0, 2030:1, 4050:0, 6070:0})
train['NEW_Age_4050'] = train['NEW_Age'].map({10:0, 2030:0, 4050:1, 6070:0})
train['NEW_Age_6070'] = train['NEW_Age'].map({10:0, 2030:0, 4050:0, 6070:1})
test['NEW_Age_010'] = test['NEW_Age'].map({10:1, 2030:0, 4050:0, 6070:0})
test['NEW_Age_2030'] = test['NEW_Age'].map({10:0, 2030:1, 4050:0, 6070:0})
test['NEW_Age_4050'] = test['NEW_Age'].map({10:0, 2030:0, 4050:1, 6070:0})
test['NEW_Age_6070'] = test['NEW_Age'].map({10:0, 2030:0, 4050:0, 6070:1})

In [110]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,NEW_Embarked,NEW_Age,NEW_Sex,NEW_Embarked_C,NEW_Embarked_Q,NEW_Embarked_S,NEW_Age_010,NEW_Age_2030,NEW_Age_4050,NEW_Age_6070
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,...,S,2030.0,0.0,0,0,1,0,1,0,0
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,...,C,2030.0,NaN,1,0,0,0,1,0,0
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,...,S,2030.0,NaN,0,0,1,0,1,0,0
3,1,1,female,35.0,1,0,113803,53.1000,C123,S,...,S,2030.0,NaN,0,0,1,0,1,0,0
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S,...,S,2030.0,0.0,0,0,1,0,1,0,0


In [213]:
test.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin',
       'Embarked', 'NEW_Cabin', 'NEW_Age', 'NEW_Fare', 'NEW_Sex',
       'NEW_Embarked_C', 'NEW_Embarked_Q', 'NEW_Embarked_S', 'NEW_Age_010',
       'NEW_Age_2030', 'NEW_Age_4050', 'NEW_Age_6070', 'NEW_Pclass_1',
       'NEW_Pclass_2', 'NEW_Pclass_3'],
      dtype='object')

In [214]:
train_X = train[['SibSp', 'Parch', 'Fare', 'NEW_Cabin', 'NEW_Sex','NEW_Embarked_C', 'NEW_Embarked_Q', 'NEW_Embarked_S', 
                 'NEW_Age_010', 'NEW_Age_2030', 'NEW_Age_4050', 'NEW_Age_6070', 'NEW_Pclass_1', 'NEW_Pclass_2', 'NEW_Pclass_3']]
train_Y = train['Survived']
test_X = test[['SibSp', 'Parch', 'Fare', 'NEW_Cabin', 'NEW_Sex', 'NEW_Embarked_C', 'NEW_Embarked_Q', 'NEW_Embarked_S', 
               'NEW_Age_010', 'NEW_Age_2030', 'NEW_Age_4050', 'NEW_Age_6070', 'NEW_Pclass_1', 'NEW_Pclass_2', 'NEW_Pclass_3']]

모델링(stacking by randomForest, bagging, boosting) - validation

In [257]:
# train - validation split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train_val_X, validation_X, train_val_Y, validation_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=12)

In [258]:
# randomforest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=2, min_samples_leaf=1, 
                                  min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                                  min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, 
                                  oob_score=False, random_state=None, verbose=0, warm_start=False, class_weight=None)
model_rf.fit(train_val_X, train_val_Y)
pred_val_rf = model_rf.predict(validation_X)
accuracy_score(validation_Y, pred_val_rf)

0.7761194029850746

In [259]:
# bagging
from sklearn.ensemble import BaggingClassifier
model_bag = BaggingClassifier(base_estimator=None, n_estimators=200, max_samples=500, max_features=10, bootstrap=True, 
                              bootstrap_features=False, oob_score=False, warm_start=False, random_state=None, verbose=0)
model_bag.fit(train_val_X, train_val_Y)
pred_val_bag = model_bag.predict(validation_X)
accuracy_score(validation_Y, pred_val_bag)

0.7910447761194029

In [260]:
# boosting
from sklearn.ensemble import GradientBoostingClassifier
model_boost = GradientBoostingClassifier(loss='deviance', learning_rate=0.401, n_estimators=100, subsample=1.0, 
                                         criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, 
                                         min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
                                         min_impurity_split=None, init=None, random_state=None, max_features=None, 
                                         verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto')
model_boost.fit(train_val_X, train_val_Y)
pred_val_boost = model_boost.predict(validation_X)
accuracy_score(validation_Y, pred_val_boost)

0.7910447761194029

In [268]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha=1.0, fit_intercept=True, normalize=False, precompute=False, copy_X=True, max_iter=1000, 
                    tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
model_lasso.fit(train_val_X, train_val_Y)
pred_val_lasso = model_lasso.predict(validation_X)
pred_val_lasso
#accuracy_score(validation_Y, pred_val_lasso)

TypeError: type numpy.ndarray doesn't define __round__ method

In [256]:
pred_val_all = (pred_val_rf + pred_val_bag + pred_val_boost)//3
accuracy_score(validation_Y, pred_val_all)

0.7761194029850746

모델링(stacking by randomForest, bagging, boosting) - test

In [263]:
model_rf.fit(train_X, train_Y)
model_bag.fit(train_X, train_Y)
model_boost.fit(train_X, train_Y)
pred_rf = model_rf.predict(test_X)
pred_bag = model_bag.predict(test_X)
pred_boost = model_boost.predict(test_X)
pred_all = (pred_rf + pred_bag + pred_boost)//3

In [264]:
pd.DataFrame(pred_all).to_csv("submission2.csv")